- get rain data and visualise for overview <font color='orange'>&#9728;</font>
- cut and split data to reasonable train, val, test sets <font color='orange'>&#9728;</font>
- set up first generator and discriminator and overfit, play around with it, get first results (_November/December_)
- train on small training set and improve losses, accuracy, results (_December_)
- experiment with temporal discriminator (_December/January_)
- experiment with wind data (_January_)

## Imports and settings

In [ ]:
import src

In [ ]:
from os import *
import numpy as np
import sys
import re
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')

In [ ]:
#forces CPU usage
environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
environ["CUDA_VISIBLE_DEVICES"] = "-1" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

______________________________________-
## Data preprocessing

### Rain measurements
Measurements are downloaded from the DWD (German weather service) open data server: __ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/__<br>
I'm working with the data of August 2010 (based on [this](https://tradingeconomics.com/germany/precipitation)), so I have downloaded this: __ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/radolan/historical/asc/2010/RW-201008.tar__<br>
_DWD manual:_<br>
__ftp://ftp-cdc.dwd.de/pub/CDC/grids_germany/hourly/radolan/historical/asc/BESCHREIBUNG_gridsgermany_hourly_radolan_historical_asc_de.pdf__<br><br>
This contains radar maps recorded in every hour. Each map has a resolution of $900\times900$ pixels and each pixel corresponds to an $1\,km\times1\,km$ area in reality. Pixel values are the precipitation height in $0.1\,mm$.
Below I'm importing the data of this as a series of numpy arrays and plot them to see the acual radar map. The _sys.path[0]_ is the path on my computer and it can be different for you.

In [ ]:
inputs = src.get_data(sys.path[0]+"/rain", total_length=101)

In [ ]:
images = src.generate_datasets(inputs, n=10, size=64, length=2, normalize=True, split=(6,2,2))

In [ ]:
images.keys()

In [ ]:
#low_res_train = images["low_res_train"]
#low_res_xval = images["low_res_xval"]
#low_res_test = images["low_res_test"]
#overfit = np.reshape(images['images'],np.shape(images['images'])+(1,))
train = np.reshape(images["train"],np.shape(images["train"])+(1,))
xval = np.reshape(images["xval"],np.shape(images["xval"])+(1,))
test = np.reshape(images["test"],np.shape(images["test"])+(1,))
print(f"Training data: {train.shape}\nValidation data: {xval.shape}\nTest data: {test.shape}")

In [ ]:
%matplotlib inline
src.visualise_data(xval[:,:,:,:,0])

# Networks

Upsampler

In [ ]:
model=keras.Sequential()

init = keras.layers.Input(shape=(1,32,32))
#32
UpSamp1 = keras.layers.UpSampling2D(size=(2, 2), data_format="channels_first")(init)
Conv1   = keras.layers.Conv2D(filters=32,kernel_size=(4,4),strides=(1,1),padding="same", data_format="channels_first")(UpSamp1)
Lr1     = keras.layers.LeakyReLU(alpha=0.0)(Conv1)
#64
UpSamp2 = keras.layers.UpSampling2D(size=(2, 2), data_format="channels_first")(Lr1)
Conv2   = keras.layers.Conv2D(filters=32,kernel_size=4,strides=1,padding="same", data_format="channels_first")(UpSamp2)
Lr2     = keras.layers.LeakyReLU(alpha=0.0)(Conv2)

Conv3   = keras.layers.Conv2D(filters=16,kernel_size=4,strides=1,padding="same", data_format="channels_first")(Lr2)
Lr3     = keras.layers.LeakyReLU(alpha=0.0)(Conv3)

Conv4   = keras.layers.Conv2D(filters=8,kernel_size=(4,4),strides=(1,1),padding="same",
                              activation = 'relu', data_format="channels_first")(Lr3)
Conv5   = keras.layers.Conv2D(filters=1,kernel_size=(4,4),strides=(1,1),padding="same",
                              activation = 'elu', data_format="channels_first")(Conv4)

model = keras.models.Model(inputs=init, outputs=Conv5)
model.summary()

In [ ]:
#overfit
overfit = np.expand_dims(low_res_train[0],axis=0)
overfit_label = np.expand_dims(train[0],axis=0)
np.shape(overfit)

In [ ]:
#train the model
model.compile(optimizer=tf.train.AdamOptimizer(learning_rate = 0.0001),loss='mean_absolute_error', metrics=['accuracy']) 
model.fit(overfit, overfit_label,
          validation_data=(overfit,overfit_label),
          batch_size = 1,
          epochs=350,
          shuffle = True)

In [ ]:
hist = model.history
hist.history.keys()

In [ ]:
plt.plot(hist.history['loss'],)
plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')

In [ ]:
#test
predictions = model.predict(low_res_test, batch_size=10)
truth       = test

In [ ]:
error_images, error_vals = src.error_distribution(truth,predictions)

In [ ]:
src.upsample_plotter(np.linspace(1,10,10),(truth, low_res_test, predictions, error_images))

# LSTM generator

In [ ]:
#batch_size=10
lstm_model = keras.Sequential()
# define CNN model
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(
    filters=8, kernel_size=5, strides=2, padding='same', activation='relu')))
lstm_model.add(keras.layers.BatchNormalization())

lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(
    filters=16, kernel_size=5, strides=2, padding='same', activation='relu')))
lstm_model.add(keras.layers.BatchNormalization())

lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(
    filters=32, kernel_size=5, strides=2, padding='same', activation='relu')))
lstm_model.add(keras.layers.BatchNormalization())

lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(
    filters=64, kernel_size=5, strides=2, padding='same', activation='relu')))
lstm_model.add(keras.layers.BatchNormalization())

lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(
    filters=128, kernel_size=5, strides=4, padding='same', activation='tanh')))
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Flatten()))

#LSTM
lstm_model.add(keras.layers.LSTM(units=128, return_sequences=True))

#upconv
lstm_model.add(keras.layers.Reshape((1,1,1,128)))
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2DTranspose(
    filters=64,kernel_size=5,strides=4,padding='same', activation='relu')))
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2DTranspose(
    filters=32,kernel_size=5,strides=2,padding='same', activation='relu')))
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2DTranspose(
    filters=16,kernel_size=5,strides=2,padding='same', activation='relu')))
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2DTranspose(
    filters=8,kernel_size=5,strides=2,padding='same', activation='relu')))
lstm_model.add(keras.layers.TimeDistributed(keras.layers.Conv2DTranspose(
    filters=1,kernel_size=5,strides=2,padding='same', activation='relu')))

lstm_model.build((None,1,)+train.shape[2:])
lstm_model.summary()

In [ ]:
lstm_train     = np.reshape(train[:,0,:,:,:],((train.shape[0],1,)+train.shape[2:]))
lstm_truth     = np.reshape(train[:,1,:,:,:],((train.shape[0],1,)+train.shape[2:]))
lstm_val       = np.reshape(xval[:,0,:,:,:],((xval.shape[0],1,)+xval.shape[2:]))
lstm_val_truth = np.reshape(xval[:,1,:,:,:],((xval.shape[0],1,)+xval.shape[2:]))
print(lstm_train.shape,"\n",lstm_truth.shape,"\n",lstm_val.shape,"\n",lstm_val_truth.shape)
lstm_test      = np.reshape(test[:,0,:,:,:],((test.shape[0],1,)+test.shape[2:]))
lstm_test_truth = np.reshape(test[:,1,:,:,:],((test.shape[0],1,)+test.shape[2:]))

In [ ]:
overfit_train = np.reshape(train[3,0,:,:,:],((1,1,)+train.shape[2:]))
overfit_truth = np.reshape(train[3,1,:,:,:],((1,1,)+train.shape[2:]))

In [ ]:
overfit_truth.shape

In [ ]:
lstm_model.compile(optimizer=tf.train.AdamOptimizer(learning_rate = 0.0001),loss="mean_absolute_error", metrics=[src.relative_error_tensor]) 
#train the model
lstm_model.fit(overfit_train, overfit_truth,
               validation_data=(lstm_val, lstm_val_truth),
               batch_size = 1,
               epochs=500,
               shuffle = True)

In [ ]:
hist = lstm_model.history
hist.history.keys()

In [ ]:
plt.plot(hist.history['loss'],)
plt.plot(hist.history['val_loss'])
plt.grid()
plt.legend(['training', 'validation'], loc='upper left')
plt.xlabel("Epoch")
plt.ylabel("Loss")
#plt.yscale("log")
plt.savefig('training.png')

In [ ]:
#test
predictions = lstm_model.predict(overfit_train, batch_size=1)
truth       = overfit_truth
truth.shape

In [ ]:
args = src.arg_getter(truth, predictions)
args

In [ ]:
error_images, error_vals, error_means = src.error_distribution(truth,predictions)

In [ ]:
src.result_plotter([0], (overfit_train[:,:,:,:,0], truth[:,:,:,:,0], predictions[:,:,:,:,0], error_images[:,:,:,:,0]))

## Discriminator

In [ ]:
D = keras.Sequential()
depth = 2
dropout = 0.4

conv1 = keras.layers.Conv2D(filters=depth*1 ,kernel_size=10, strides=5, input_shape=(900,900,1), padding='same')
relu1 = keras.layers.LeakyReLU(alpha=0.2)
dropout1 = keras.layers.Dropout(dropout)

conv2 = keras.layers.Conv2D(filters=depth*2, kernel_size=10, strides=5, padding='same')
relu2 = keras.layers.LeakyReLU(alpha=0.2)
dropout2 = keras.layers.Dropout(dropout)

conv3 = keras.layers.Conv2D(filters=depth*4, kernel_size=6, strides=2, padding='same')
relu3 = keras.layers.LeakyReLU(alpha=0.2)
dropout3 = keras.layers.Dropout(dropout)

conv4 = keras.layers.Conv2D(filters=depth*8, kernel_size=2, strides=2, padding='same')
relu4 = keras.layers.LeakyReLU(alpha=0.2)
dropout4 = keras.layers.Dropout(dropout)

# Out: 1-dim probability
flatten = keras.layers.Flatten()
fcl1 = keras.layers.Dense(1)
sig1 = keras.layers.Activation('sigmoid')
#summary()
D.add(conv1)
D.add(relu1)
D.add(dropout1)
D.add(conv2)
D.add(relu2)
D.add(dropout2)
D.add(conv3)
D.add(relu3)
D.add(dropout3)
D.add(conv4)
D.add(relu4)
D.add(dropout4)
D.add(flatten)
D.add(fcl1)
D.add(sig1)
D.summary()

In [ ]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.0001, decay=3e-8)
AM = keras.Sequential()
AM.add(G)
AM.add(D)
AM.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
AM.fit(train_val_inputs,
          train_val_targets,
          batch_size = 50,
          epochs=50,
          validation_split = 0.1,
          shuffle = True)